In [339]:
"""
Purpose: To gather the orientation preferences and the soma center data and try to find different ways to correlate

Pseudocode:
1) Find the data for the oma locations
2) Find way to extract the orientation preferences and bin them
3) Get the total distributions the orientation| preferences from inside the volume
4) Start with a given neuron: 
a. Do radius testing:
    - plot the overall distribution of circular orinetation difference from that neuron for the whole volume
    - gather all of the neurons that are within a certain radius
    - plot the distirbutions of circular orinetation difference from that neuron for neurons within that radius
    - continue and vary the size of the radius
b. Columns of X, Y, Z (same as radius testing)
    - need to vary the widths and sizes of the columns like varying the size of the radius

"""

'\nPurpose: To gather the orientation preferences and the soma center data and try to find different ways to correlate\n\nPseudocode:\n1) Find the data for the oma locations\n2) Find way to extract the orientation preferences and bin them\n3) Get the total distributions the orientation| preferences from inside the volume\n4) Start with a given neuron: \na. Do radius testing:\n    - plot the overall distribution of circular orinetation difference from that neuron for the whole volume\n    - gather all of the neurons that are within a certain radius\n    - plot the distirbutions of circular orinetation difference from that neuron for neurons within that radius\n    - continue and vary the size of the radius\nb. Columns of X, Y, Z (same as radius testing)\n    - need to vary the widths and sizes of the columns like varying the size of the radius\n\n'

In [340]:
import numpy as np
import datajoint as dj

In [ ]:
pinky = dj.create_virtual_module("pinky","microns_pinky")
schema = dj.schema("microns_pinky")
pinky_nda = dj.create_virtual_module('pinky_nda', 'microns_pinky_nda')
radtune = dj.create_virtual_module('pinky_radtune', 'microns_pinky_radtune')
spattune = dj.create_virtual_module('pinky_spattune', 'microns_pinky_spattune')
fc = dj.create_virtual_module('pinky_fc', 'microns_pinky_fc')

# getting the Soma Centers

In [ ]:
# locations of all of the cell somas
pinky.AllenSoma()

# GET THE SIGNIFICANTLY TUNED NEURONS

In [ ]:

attrs = ['segment_a', 'segment_b', 'connection', 'n_seg_shared']
good_rf = 'mscore > 1.25'

rf_bin_edges = np.linspace(-.7, .7, 6) #setting the bin boundaries


"""
Calculate the bins and centers for the orientational preference
"""
#defines how to find the "tuned" segments of the orientation
dori_resolution = np.pi / 2 / 4 #setting the width size of the bins to be 22.5 degrees
rad2deg = 180/np.pi
bin_edges = np.linspace(0, np.pi,9) #creates 8 bins between boundaries between 0 and pi


#sets the threshold for the 
stat_choice = ["synapse_vol_density_pearson"]

ori_confidence=0.5
von_p_value=0.05

rf_bin_edges=rf_bin_edges
ori_bin_edges=bin_edges
n_seg_shared_threshold = 10
n_seg_shared_converted_threshold = 10
rf_threshold=1.45

In [ ]:
# get the significantly tuned neurons: 


synapse = (pinky.Synapse - pinky.SynapseExclude) & pinky.CurrentSegmentation
segment = (pinky.Segment - pinky.SegmentExclude) & pinky.CurrentSegmentation
soma = (pinky.AllenSoma - pinky.SegmentExclude) & pinky.CurrentSegmentation
soma = soma & "cell_class='excitatory'"

"""calculate the specifics for the bins based on the bin edges given"""
###Receptive Field
#calculates the centers of the bins used for the receptive field
cbin_centers = np.hstack((np.nan, np.round((rf_bin_edges[1:] + rf_bin_edges[:-1])/2, decimals=2), np.nan))
#creates labels for the receptive field edges
cbin_labels = ['[{:.1f},{:.1f}]'.format(*a) for a in zip(rf_bin_edges[:-1], rf_bin_edges[1:])]
cbin_labels

###Orientation
rad2deg = 180/np.pi
# turns all of the bin edges into degrees with no decimal
be = list(['{:.0f}'.format(ee) for ee in [np.round(e * rad2deg) for e in ori_bin_edges]])
#creates the bin boundary markings with [low - high]
bin_labels = list(zip(be[:-1], be[1:]))

#calculates the bin centers
bin_centers = np.round((ori_bin_edges[1:] + ori_bin_edges[:-1])/2 * rad2deg, decimals=2)

"""
Gets the segments with the acceptable rf thresholds
"""

# functional metrics for each functional soma pair (restricting by spattune fitting significance)

sig_units_rf = spattune.BestSTA.Loc & 'sta_snr > ' + str(rf_threshold) & segment
sig_unit_pairs_rf = (sig_units_rf.proj(segment_id1 = 'segment_id') * 
                  sig_units_rf.proj(segment_id2 = 'segment_id')) & 'segment_id1 < segment_id2'

print("Number of significant receptive field neurons = " + str(len(sig_units_rf)))

rf_table = (spattune.BestSTACorr.proj(rf_corr_coef="union_corr_r2",
                          segment_a="segment_id1",segment_b="segment_id2")
                          & sig_units_rf.proj(segment_a="segment_id") &
                          sig_units_rf.proj(segment_b="segment_id"))
print("Length of rf correlation = " + str(len(rf_table)))

    


In [ ]:
tuned = 'confidence > ' + str(ori_confidence)
#get the significantly tuned segments
sig_units_op = radtune.BestVonFit.Unit & 'von_p_value <= ' + str(von_p_value) & tuned & segment
print("Number of significanlty orientationally tuned neurons = " + str(len(sig_units_op)))


In [ ]:
#gets the significantly tuned neurons and their differences in combinational pairs 
sig_unit_pairs_op = (radtune.BestVonCorr() & sig_units_op.proj(segment_id1="segment_id") 
                 & sig_units_op.proj(segment_id2="segment_id")).proj("diff_pref_ori")
sig_unit_pairs_op = sig_unit_pairs_op.proj(segment_a="segment_id1",
                                           segment_b="segment_id2",
                                           dori="diff_pref_ori")
sig_unit_pairs_op

In [ ]:
# checked correct number of pairwise
n_units = 299
(n_units)*(n_units-1)/2

In [ ]:
sig_units_op

In [ ]:
seg_ids, thetas, amps = sig_units_op.fetch("segment_id","thetas","amps")
direction_preference = [thetas[index][np.argmax(amps[index])] for index in range(0,len(thetas))]



In [ ]:
orientation_preference = [k if k<np.pi else k - np.pi for k in direction_preference]
len(orientation_preference)

# get the soma centers and save offline

In [ ]:
pinky.AllenSoma()

In [ ]:
tuned_soma_center_cells = pinky.AllenSoma() & sig_units_op.proj() & "cell_class='excitatory'"
tuned_soma_center_cells

In [ ]:
soma_cell_data = tuned_soma_center_cells.fetch(as_dict=True)

In [ ]:
np.savez("orientation_center_data.npz",soma_cell_data = soma_cell_data,
         orientation_preference = orientation_preference,
         seg_ids = seg_ids
        )

# ********************load the data from local file******************

In [ ]:
import numpy as np
import datajoint as dj

orientation_center_data = np.load("orientation_center_data.npz",allow_pickle=True)
orientation_center_data.files
soma_cell_data = orientation_center_data["soma_cell_data"] #stores the soma id, segment_id and the x,y,z coordinates as well as the cell_class = excitatory
orientation_preference = orientation_center_data["orientation_preference"] #the orientation preferences in radians that correspond tot he 
seg_ids = orientation_center_data["seg_ids"] #the segment ids for the orientation preferences

#build a lookup dictionary for segment ids to orientation
seg_id_to_orientation = dict([(seg_ids[k],orientation_preference[k]) for k in range(0,len(seg_ids))])
#seg_id_to_orientation

# Where to start in offline version:

In [ ]:

attrs = ['segment_a', 'segment_b', 'connection', 'n_seg_shared']
good_rf = 'mscore > 1.25'

rf_bin_edges = np.linspace(-.7, .7, 6) #setting the bin boundaries


"""
Calculate the bins and centers for the orientational preference
"""
#defines how to find the "tuned" segments of the orientation
dori_resolution = np.pi / 2 / 4 #setting the width size of the bins to be 22.5 degrees
rad2deg = 180/np.pi
bin_edges = np.linspace(0, np.pi,9) #creates 8 bins between boundaries between 0 and pi


#sets the threshold for the 
stat_choice = ["synapse_vol_density_pearson"]

ori_confidence=0.5
von_p_value=0.05

rf_bin_edges=rf_bin_edges
ori_bin_edges=bin_edges
n_seg_shared_threshold = 10
n_seg_shared_converted_threshold = 10
rf_threshold=1.45

###Orientation
rad2deg = 180/np.pi
# turns all of the bin edges into degrees with no decimal
be = list(['{:.0f}'.format(ee) for ee in [np.round(e * rad2deg) for e in ori_bin_edges]])
#creates the bin boundary markings with [low - high]
bin_labels = list(zip(be[:-1], be[1:]))

#calculates the bin centers
bin_centers = np.round((ori_bin_edges[1:] + ori_bin_edges[:-1])/2 * rad2deg, decimals=2)

In [ ]:
"""
The cdiff computes the angle 
that the second angle has to move in order to meet
up with the first angle

Where if you have to move counterclockwise --> positive
If you have to move clockwise --> negative

"""

import pycircstat as pycs

def center_angle(angle):
    return (angle + np.pi) % (2*np.pi) - np.pi

def cdiff(alpha, beta):
    """
    Difference between pairs :math:`x_i-y_i` around the circle,
    computed efficiently.
    :param alpha:  sample of circular random variable
    :param beta:   sample of circular random variable
    :return: distance between the pairs
    """
    return center_angle(alpha - beta)
def cdiff_orientation(alpha,beta):
    return center_angle(alpha*2 - beta*2)/2

In [ ]:
"""
Testing out the circular functions

functions that will be helpful to use from pycircstat: 
mean
pairwise_cdiff
cdiff: Difference between pairs
(angle + np.pi) % (2*np.pi) - np.pi
median

"""
# # get the circular angular difference between the two
# segment_1_ori = orientation_preference[0]
# segment_2_ori = orientation_preference[1]
# segment_1_ori = 1.4
# segment_2_ori = 1.39
# cdiff_orientation(segment_1_ori,segment_2_ori)

# Visualizing the Data (for the whole volume): 

In [ ]:
# fig = plt.figure(figsize=(20,20)) #2nd does the height
# n_subplot = 4


# """ horizontal implementation"""
# currnet_subplot = 1
# ax = plt.subplot(1,n_subplot,currnet_subplot)
# currnet_subplot += 1
# ax = plt.subplot(1,n_subplot,currnet_subplot)
# currnet_subplot += 1
# ax = plt.subplot(1,n_subplot,currnet_subplot)
# currnet_subplot += 1
# ax = plt.subplot(1,n_subplot,currnet_subplot)

# """ vertical implementation"""
# currnet_subplot = 1
# ax = plt.subplot(n_subplot,1,currnet_subplot)
# currnet_subplot += 1
# ax = plt.subplot(n_subplot,1,currnet_subplot)
# currnet_subplot += 1
# ax = plt.subplot(n_subplot,1,currnet_subplot)
# currnet_subplot += 1
# ax = plt.subplot(n_subplot,1,currnet_subplot)

# # g.set_yticklabels(['{}°-{}°'.format(*a) for a in bin_labels], rotation=0)
# # #sets the rf labels and rotates them slightly
# # g.set_xticklabels(cbin_labels, rotation=30)
# # #inverts the y axis so that they are going up in value as graph goes up
# # g.invert_yaxis()
# # #sets no tick marks
# # g.tick_params(length=0)
# # #set labels
# # g.set_xlabel(r'$\rho$(RF$_1$, RF$_2$)')
# # g.set_ylabel(r'$\langle$Number of pairs in bin$\rangle$')

# # #Add title based on all the parameters

# # # g.set_title('add title here')
# # g.set_title('Number of pairs in bins')

In [ ]:
# set up bins for difference in prefered orientation
n_bins = 20
lower_bin_bound = 0
upper_bin_bound = np.pi
rad2deg = 180/np.pi
ori_edges = np.linspace(lower_bin_bound, upper_bin_bound, n_bins+1)
oe = list(['{:.0f}'.format(ee) for ee in [np.round(e * rad2deg) for e in ori_edges]])
ori_labels = list(zip(oe[:-1], oe[1:]))
ori_centers = np.round((ori_edges[1:] + ori_edges[:-1])/2 * rad2deg, decimals=2) 
print("ori_centers = " + str(ori_centers))

bin_centers = ori_centers
ori_bin_edges=ori_edges
binned_ori_preferences = bin_centers[(np.digitize(orientation_preference, ori_bin_edges))-1]
raw_orientation_degrees = (np.array(orientation_preference)*rad2deg).astype("int")

#np.vstack([binned_ori_preferences.T,raw_orientation_degrees.T]).T
#cdf['bdori'] = bin_centers[(np.digitize(np.abs(cdf["dori"]), ori_bin_edges))-1]

In [ ]:
"""
For the whole volume: Gets all the bins of the orientation and how many neurons in each group
"""

from collections import Counter
my_counter = Counter(binned_ori_preferences)
#print(my_counter)
binned_angles = np.array(list(my_counter.keys()))
binned_angles_histogram = np.array(list(my_counter.values()))
print(binned_angles,binned_angles_histogram)

In [ ]:
#plot as distribution of the orientations in the whole volume
import matplotlib.pyplot as plt

plt.figure()
plt.bar(binned_angles,binned_angles_histogram,width=rad2deg*(ori_edges[1]-ori_edges[0])-1)
#plot as radians
plt.figure()
plt.bar(binned_angles/rad2deg,binned_angles_histogram,width=ori_edges[1]-ori_edges[0]-0.1)

# Trying to get circular plots of the distribution

In [ ]:
"""
Visuallize the distribution of the orientation preferences in the volume on a circular plot
https://stackoverflow.com/questions/22562364/circular-histogram-for-python
"""

import numpy as np
import matplotlib.pyplot as plt

def plot_circular_distribution(theta,
                               radii,
                               width=-1,
                               bottom = 8,
                               max_height = 4):
    N = len(radii)
    #theta = np.linspace(0.0, 2 * np.pi, N, endpoint=False)
    #radii = max_height*np.random.rand(N)
    if width == -1:
        width = (2*np.pi) / N

    ax = plt.subplot(111, polar=True)
    bars = ax.bar(theta, radii, width=width, bottom=bottom)

    # Use custom colors and opacity
    for r, bar in zip(radii, bars):
        bar.set_facecolor(plt.cm.jet(r / 10.))
        bar.set_alpha(0.8)
    #ax.set_yticks([0,45,90,135,180])
    """
    ax.set_xticks(np.pi/180. * np.linspace(0,  180, 8, endpoint=False))
    ax.set_thetalim(0,np.pi)
    """
    ax.set_xticklabels(np.linspace(0,  180, 8, endpoint=False))
    plt.show()

#function will bin the data
plot_circular_distribution(binned_angles/rad2deg*2,binned_angles_histogram,width=(ori_edges[1]-ori_edges[0])*2-0.1)

# Calculating the von misses fit

In [ ]:


# from scipy.stats import vonmises
# import matplotlib.pyplot as plt
# fig, ax = plt.subplots(1, 1)

# kappa = 3.99
# mean, var, skew, kurt = vonmises.stats(kappa, moments='mvsk')

# x = np.linspace(vonmises.ppf(0.01, kappa),
#                 vonmises.ppf(0.99, kappa), 100)
# ax.plot(x, vonmises.pdf(x, kappa),
#        'r-', lw=5, alpha=0.6, label='vonmises pdf')

# -- Step 2: Get distribution stats of the circular data --

In [ ]:
"""practices getting the circular statistics for orientation that only ranges from 0 - 180"""

binned_ori_preferences 
doubled_orientation_preference = np.array(orientation_preference)*2
np.max(doubled_orientation_preference),np.min(doubled_orientation_preference)

"""mean
pairwise_cdiff
cdiff: Difference between pairs
(angle + np.pi) % (2*np.pi) - np.pi
median
"""
print("mean = " + str(pycs.mean(doubled_orientation_preference)/2))
print("variance = " + str(pycs.var(doubled_orientation_preference)/4))
print("std deviation = " + str(pycs.std(doubled_orientation_preference)/2))


# Finding vector between soma ceneters and finding true distance:

In [ ]:
"""Turns the downloaded table of soma data into a dictionary with the segment_id as the key"""
soma_cell_data
soma_cell_location_dict = dict([(k["segment_id"],dict(location=np.array([k["soma_x"],k["soma_y"],k["soma_z"]]),x_loc=k["soma_x"],y_loc=k["soma_y"],z_loc=k["soma_z"])) for k in soma_cell_data])
#soma_cell_location_dict

In [ ]:
""" Demonstrates how to get the distance between the two somas"""
seg_1 = 648518346341371119
seg_2 = 648518346349470171
location_1 = soma_cell_location_dict[seg_1]["location"]
location_2 = soma_cell_location_dict[seg_2]["location"]
print(location_1,location_2)
print(np.linalg.norm(location_1-location_2))


In [ ]:
"""create a 2D array that has the distances between each cell pair
Includes the:
1) overall distance 
2) Distance along each axis
"""
from tqdm import tqdm

soma_distance_array = dict()

for seg_1 in tqdm(soma_cell_location_dict.keys()):
    for seg_2 in soma_cell_location_dict.keys():
        location_1 = soma_cell_location_dict[seg_1]["location"]
        location_2 = soma_cell_location_dict[seg_2]["location"]
        if seg_1 not in soma_distance_array.keys():
            soma_distance_array[seg_1] = dict()
        soma_distance_array[seg_1][seg_2] = dict(distance=np.linalg.norm(location_1-location_2),
                                                 x_distance = location_1[0] - location_2[0], 
                                                 y_distance = location_1[1] - location_2[1], 
                                                 z_distance = location_1[2] - location_2[2])
        

In [ ]:
"""
Demonstrates how to get a subgroup that is either 
1) within a certian radius of target neuron
2) within a specified column with the target neuron
"""



target_neuron = 648518346341371119
target_neuron_orientation_preferene = seg_id_to_orientation[target_neuron]

"""
Example of how to find neurons within a certain radius
"""
total_neurons_list = list(soma_cell_location_dict.keys())
total_neurons_list.remove(target_neuron)

circular_radius = 15000
circular_radius_neurons = [k for k in total_neurons_list if abs(soma_distance_array[target_neuron][k]["distance"]) < circular_radius ]
print("len(circular_radius_neurons) = " + str(len(circular_radius_neurons)))

"""
Example of how to find neurons within z column
"""

box_width = 10000
box_height = 2000
width_axis = "x_distance"
height_axis = "y_distance"

column_neurons = [k for k in total_neurons_list if ((abs(soma_distance_array[target_neuron][k][width_axis]) < box_width)
                                                                      and (abs(soma_distance_array[target_neuron][k][height_axis]) < box_height))]

print("len(column_neurons) = " + str(len(column_neurons)))
print(column_neurons)


In [ ]:
"""
Get the orientation distribution and difference in orientation distributions for a target neuron and it's restricted neighbors
"""
global_neuron_group = total_neurons_list = list(soma_cell_location_dict.keys())
print("global_neuron_group = " + str(len(global_neuron_group)))
restricted_neuron_group = circular_radius_neurons
print("restricted_neuron_group = " + str(len(restricted_neuron_group)))

#get the orientations of these groups
restricted_neuron_group_orientations = [seg_id_to_orientation[k] for k in restricted_neuron_group]

global_neuron_group_orientations = [seg_id_to_orientation[k] for k in global_neuron_group]


#send these orientations to be function that will display the linear and/or circular distribution and/or total distribution
# Also want to compute and print out stats:


In [ ]:
def bin_orientation_data(orientation_preference,
                         n_bins = 20,
                        lower_bin_bound = 0,
                         upper_bin_bound = np.pi):
    """
    Functions that will do binning of data
    Bases on the number of bins, lower bound and upper bound specified
    """
    
    rad2deg = 180/np.pi
    ori_edges = np.linspace(lower_bin_bound, upper_bin_bound, n_bins+1)
    oe = list(['{:.0f}'.format(ee) for ee in [np.round(e * rad2deg) for e in ori_edges]])
    ori_labels = list(zip(oe[:-1], oe[1:]))
    ori_centers = np.round((ori_edges[1:] + ori_edges[:-1])/2 * rad2deg, decimals=2) 

    bin_centers = ori_centers
    ori_bin_edges=ori_edges
    binned_ori_preferences = bin_centers[(np.digitize(orientation_preference, ori_bin_edges))-1]
    raw_orientation_degrees = (np.array(orientation_preference)*rad2deg).astype("int")
    
    from collections import Counter
    my_counter = Counter(binned_ori_preferences)
    #print(my_counter)
    binned_angles = np.array(list(my_counter.keys()))
    binned_angles_histogram = np.array(list(my_counter.values()))
    
    return binned_angles,binned_angles_histogram

def plot_circular_distribution_lite(theta,
                               radii,
                                    ax,
                                    n_bins,
                               width=-1,
                               bottom = 8):
    
    """
    will plot the circulation distribution given the 
    1) theta (x values)
    2) radii (y values)
    
    Other parameters passed: 
    0) axis to plot it on
    1) number of bins that could possibly be plotted
    
    optional
    1) Width of each of the bins (else it is calculated)
    2) bottom: The offset where the histogram starts
    
    
    
    
    """
    N = n_bins
    
    if width == -1:
        width = (2*np.pi) / N

    #ax = plt.subplot(111, polar=True)
    bars = ax.bar(theta, radii, width=width, bottom=bottom)

    # Use custom colors and opacity
    for r, bar in zip(radii, bars):
        bar.set_facecolor(plt.cm.jet(r / 10.))
        bar.set_alpha(0.8)
    #ax.set_yticks([0,45,90,135,180])
    """
    ax.set_xticks(np.pi/180. * np.linspace(0,  180, 8, endpoint=False))
    ax.set_thetalim(0,np.pi)
    """
    ax.set_xticklabels(np.linspace(0,  180, 8, endpoint=False))
    #plt.show()

def graph_binned_orientation_data(binned_angles,
                                binned_angles_histogram,
                                  neuron_id,
                                  title,
                                  n_bins,
                                  global_binned_angles = [],
                                global_binned_angles_histogram= [],
                                 graphs_to_plot=[],
                                  figure_size = (20,20),
                                 circular_flag = True,
                                 target_neuron_orientation = -1,
                                 target_color="y"):
    """
    Will graph the linear/circular or both of the distribution of a certain group
    1) Will graph just the local group or global group alone if only
    "local" or "global" sent in graph to plot
    
    And then highlight the target neuron's orientation 
    """
    
    axis_label_size=20
    tick_label_size = 20
    title_size=30
    
    # if circular plots were also requested then adjust overall size
    if circular_flag:
        n_subplot = 2*len(graphs_to_plot)
    else:
        n_subplot = len(graphs_to_plot)
    
    width_size = 5
    #print("n_subplot = " + str(n_subplot))
    currnet_subplot = 1
    fig = plt.figure(figsize=(figure_size[0],figure_size[1]*len(graphs_to_plot)))
    fig.tight_layout()
    #fig.set_size_inches(20, 12)
    if "local" in graphs_to_plot:
        
        if circular_flag:
            ax = plt.subplot(n_subplot,2,currnet_subplot)
            #graphing the linear graph
            plt.bar(binned_angles,binned_angles_histogram,width=width_size)
            if target_neuron_orientation != -1:
                plt.bar(target_neuron_orientation,[0.5],width=width_size,color=target_color)
#                 print(" target_neuron_orientation[0] = " + str( target_neuron_orientation[0]))
#                 print("binned_angles = " + str(binned_angles))
#                 angle_index = np.where(binned_angles == target_neuron_orientation[0])[0]
#                 if len(angle_index) == 0:
#                     target_neuron_height = 0.5
#                 else:
#                     target_neuron_height = binned_angles_histogram[angle_index]
#                 print("angle_index = " + str(angle_index))
#                 print("target_neuron_orientation/rad2deg = " + str(target_neuron_orientation/rad2deg))
#                 print("[target_neuron_height] =  " + str([target_neuron_height]))
#                 plt.bar(target_neuron_orientation/rad2deg,[target_neuron_height],width=width_size,color=target_color)
                #             ax.set(
#                    title=title + "_linear",
#                    ylabel='Number of Neurons',
#                    xlabel='Radians',
#                     titlesize=20,
#                     labelsize=20)
            ax.set_title(title+"_linear",fontsize=title_size)
            ax.set_xlabel("Degrees",fontsize=axis_label_size)
            ax.set_ylabel('Number of Neurons',fontsize=axis_label_size)
            ax.tick_params(axis="both",which="major",labelsize=tick_label_size)
            
            #graphing the circular graph
            ax = plt.subplot(n_subplot,2,currnet_subplot+1,polar=True)
            plot_circular_distribution_lite(binned_angles/rad2deg*2,
                                            binned_angles_histogram,
                                            ax,
                                           n_bins)
            
#             ax.set(
#                    title=title + "_circular",
#                    ylabel='Number of Neurons',
#                    xlabel='Degrees',
#                     titlesize=20,
#                     labelsize=20)
            ax.set_title(title+"_circular",fontsize=title_size)
            ax.set_xlabel("Degrees",fontsize=axis_label_size)
            ax.set_ylabel('Number of Neurons',fontsize=axis_label_size)
            ax.tick_params(axis="both",which="major",labelsize=tick_label_size)
#             ax.set_xticklabels(ax.get_xticklabels(),fontsize=tick_label_size)
#             ax.set_yticklabels(ax.get_yticklabels(),fontsize=tick_label_size)
            
            #increment the current_subplot
            currnet_subplot += 2
        else:
            ax = plt.subplot(n_subplot,1,currnet_subplot)
            #graphing the linear graph
            plt.bar(binned_angles,binned_angles_histogram,width=width_size)
            if target_neuron_orientation != -1:
                plt.bar(target_neuron_orientation,[0.5],width=width_size,color=target_color)
#             ax.set(
#                    title=title + "_linear",
#                    ylabel='Number of Neurons',
#                    xlabel='Radians',
#                     titlesize=20,
#                     labelsize=20)
            ax.set_title(title+"_linear",fontsize=title_size)
            ax.set_xlabel("Degrees",fontsize=axis_label_size)
            ax.set_ylabel('Number of Neurons',fontsize=axis_label_size)
            ax.tick_params(axis="both",which="major",labelsize=tick_label_size)
            #increment the current_subplot
            currnet_subplot += 1
    
    print("\n\n\n")
    if "global" in graphs_to_plot:
        binned_angles = global_binned_angles
        binned_angles_histogram =  global_binned_angles_histogram
        if circular_flag:
            ax = plt.subplot(n_subplot,2,currnet_subplot)
            #graphing the linear graph
            plt.bar(binned_angles,binned_angles_histogram,width=width_size)
            
#             ax.set(
#                    title=title + "_linear",
#                    ylabel='Number of Neurons',
#                    xlabel='Radians',
#                     titlesize=20,
#                     labelsize=20)
            ax.set_title(title+"_global_linear",fontsize=title_size)
            ax.set_xlabel("Degrees",fontsize=axis_label_size)
            ax.set_ylabel('Number of Neurons',fontsize=axis_label_size)
            ax.tick_params(axis="both",which="major",labelsize=tick_label_size)
            
            #graphing the circular graph
            ax = plt.subplot(n_subplot,2,currnet_subplot+1,polar=True)
            plot_circular_distribution_lite(binned_angles/rad2deg*2,
                                            binned_angles_histogram,
                                            ax,n_bins)
            
#             ax.set(
#                    title=title + "_circular",
#                    ylabel='Number of Neurons',
#                    xlabel='Degrees',
#                     titlesize=20,
#                     labelsize=20)
            ax.set_title(title+"_global_circular",fontsize=title_size)
            ax.set_xlabel("Degrees",fontsize=axis_label_size)
            ax.set_ylabel('Number of Neurons',fontsize=axis_label_size)
            ax.tick_params(axis="both",which="major",labelsize=tick_label_size)
#             ax.set_xticklabels(ax.get_xticklabels(),fontsize=tick_label_size)
#             ax.set_yticklabels(ax.get_yticklabels(),fontsize=tick_label_size)
            
            #increment the current_subplot
            currnet_subplot += 2
        else:
            ax = plt.subplot(n_subplot,1,currnet_subplot)
            #graphing the linear graph
            plt.bar(binned_angles,binned_angles_histogram,width=width_size)
            
#             ax.set(
#                    title=title + "_linear",
#                    ylabel='Number of Neurons',
#                    xlabel='Radians',
#                     titlesize=20,
#                     labelsize=20)
            ax.set_title(title+"_global",fontsize=title_size)
            ax.set_xlabel("Degrees",fontsize=axis_label_size)
            ax.set_ylabel('Number of Neurons',fontsize=axis_label_size)
            ax.tick_params(axis="both",which="major",labelsize=tick_label_size)
            #increment the current_subplot
            currnet_subplot += 1
    left = 0.125
    right = 0.9
    bottom = 0.1
    top = 0.9
    wspace = 0.2
    hspace = 0.5
    fig.subplots_adjust(left=left,bottom=bottom,right=right,
                       top=top,wspace=wspace,hspace=hspace)

binned_angles,binned_angles_histogram = bin_orientation_data(orientation_preference,
                                                         n_bins = 20,
                                                        lower_bin_bound = 0,
                                                         upper_bin_bound = np.pi)

# graph_binned_orientation_data(binned_angles,
#                                 binned_angles_histogram,
#                                   neuron_id=45,
#                               n_bins=20,
#                                   title="example_type",
#                               global_binned_angles = binned_angles,
#                                 global_binned_angles_histogram= binned_angles_histogram,
#                                  graphs_to_plot=["local","global"],
#                                  figure_size = (20,20),
#                                  circular_flag = True)

In [ ]:
current_n_bins = 20
current_lower_bin_bound = 0
current_upper_bin_bound = np.pi

binned_angles,binned_angles_histogram = bin_orientation_data(restricted_neuron_group_orientations,
                                                         n_bins = current_n_bins,
                                                        lower_bin_bound = current_lower_bin_bound,
                                                         upper_bin_bound = current_upper_bin_bound)

global_binned_angles,global_binned_angles_histogram = bin_orientation_data(global_neuron_group_orientations,
                                                         n_bins = current_n_bins,
                                                        lower_bin_bound = current_lower_bin_bound,
                                                         upper_bin_bound = current_upper_bin_bound)

#for getting the binning of the target neuron
interest_binned_angles,interest_binned_angles_histogram = bin_orientation_data([target_neuron_orientation_preferene],
                                                        n_bins = current_n_bins,
                                                        lower_bin_bound = current_lower_bin_bound,
                                                         upper_bin_bound = current_upper_bin_bound)

print(interest_binned_angles)

#gets the plots of the both the global and the subgroup while highlighting the neuron of interest
# plots both linear and circular

graph_binned_orientation_data(binned_angles,
                                binned_angles_histogram,
                                  neuron_id=45,
                                  title="example_type",
                              n_bins=current_n_bins,
                              global_binned_angles = global_binned_angles,
                                global_binned_angles_histogram= global_binned_angles_histogram,
                                 graphs_to_plot=["local","global"],
                                 figure_size = (20,20),
                                 circular_flag = True,
                                 target_neuron_orientation = interest_binned_angles)

# Find good column and radius 

In [ ]:
circular_radius = 1000

box_width = 1000

suitable_number_threshold = 10
total_neurons_in_cluster = []

suitable_number = False
while suitable_number == False:
    for target_neuron in list(soma_cell_location_dict.keys()):
        target_neuron_orientation_preferene = seg_id_to_orientation[target_neuron]
        """
        Example of how to find neurons within a certain radius
        """
        total_neurons_list = list(soma_cell_location_dict.keys())
        total_neurons_list.remove(target_neuron)

        circular_radius_neurons = [k for k in total_neurons_list if soma_distance_array[target_neuron][k]["distance"] < circular_radius ]
        #print("len(circular_radius_neurons) = " + str(len(circular_radius_neurons)))
        total_neurons_in_cluster.append(len(circular_radius_neurons))

        """
        Example of how to find neurons within x column
        """
    # evaluate if enough neurons in each cluster (if 90 percent have 10 or more)
    percentage_above_threshold = np.sum(np.array(total_neurons_in_cluster) >= suitable_number_threshold)/len(total_neurons_in_cluster)
    if percentage_above_threshold >= 0.9:
        suitable_number = True
    else:
        circular_radius += 100
        total_neurons_in_cluster = []
        
    
print("total_neurons_in_cluster = " + str(total_neurons_in_cluster))
print("Final circular_radius for circular  = " + str(circular_radius))

"""
Final size: suitable number would be 15000
"""


suitable_number_threshold = 10
box_width = 1000
width_axis = "x_distance"
height_axis = "y_distance"

total_neurons_in_cluster = []

suitable_number = False
while suitable_number == False:
    for target_neuron in list(soma_cell_location_dict.keys()):
        target_neuron_orientation_preferene = seg_id_to_orientation[target_neuron]
        """
        Example of how to find neurons within a certain radius
        """
        total_neurons_list = list(soma_cell_location_dict.keys())
        total_neurons_list.remove(target_neuron)

        column_neurons = [k for k in total_neurons_list if ((abs(soma_distance_array[target_neuron][k][width_axis]) < box_width)
                                                                      and (abs(soma_distance_array[target_neuron][k][height_axis]) < box_width))]
        #print("len(circular_radius_neurons) = " + str(len(circular_radius_neurons)))
        total_neurons_in_cluster.append(len(column_neurons))

        """
        Example of how to find neurons within x column
        """
    # evaluate if enough neurons in each cluster (if 90 percent have 10 or more)
    percentage_above_threshold = np.sum(np.array(total_neurons_in_cluster) >= suitable_number_threshold)/len(total_neurons_in_cluster)
    if percentage_above_threshold >= 0.9:
        suitable_number = True
    else:
        box_width += 50
        total_neurons_in_cluster = []

#print the suitable threshold
print("total_neurons_in_cluster Z AXIS = " + str(total_neurons_in_cluster))
print("Final box_width for circular  Z AXIS = " + str(box_width))

suitable_number_threshold = 10
box_width = 1000
width_axis = "y_distance"
height_axis = "z_distance"

total_neurons_in_cluster = []

suitable_number = False
while suitable_number == False:
    for target_neuron in list(soma_cell_location_dict.keys()):
        target_neuron_orientation_preferene = seg_id_to_orientation[target_neuron]
        """
        Example of how to find neurons within a certain radius
        """
        total_neurons_list = list(soma_cell_location_dict.keys())
        total_neurons_list.remove(target_neuron)

        column_neurons = [k for k in total_neurons_list if ((abs(soma_distance_array[target_neuron][k][width_axis]) < box_width)
                                                                      and (abs(soma_distance_array[target_neuron][k][height_axis]) < box_width))]
        #print("len(circular_radius_neurons) = " + str(len(circular_radius_neurons)))
        total_neurons_in_cluster.append(len(column_neurons))

        """
        Example of how to find neurons within x column
        """
    # evaluate if enough neurons in each cluster (if 90 percent have 10 or more)
    percentage_above_threshold = np.sum(np.array(total_neurons_in_cluster) >= suitable_number_threshold)/len(total_neurons_in_cluster)
    if percentage_above_threshold >= 0.9:
        suitable_number = True
    else:
        box_width += 50
        total_neurons_in_cluster = []

#print the suitable threshold
print("total_neurons_in_cluster X AXIS = " + str(total_neurons_in_cluster))
print("Final box_width for circular  X AXIS = " + str(box_width))

suitable_number_threshold = 10
box_width = 1000
width_axis = "x_distance"
height_axis = "z_distance"

total_neurons_in_cluster = []

suitable_number = False
while suitable_number == False:
    for target_neuron in list(soma_cell_location_dict.keys()):
        target_neuron_orientation_preferene = seg_id_to_orientation[target_neuron]
        """
        Example of how to find neurons within a certain radius
        """
        total_neurons_list = list(soma_cell_location_dict.keys())
        total_neurons_list.remove(target_neuron)

        column_neurons = [k for k in total_neurons_list if ((abs(soma_distance_array[target_neuron][k][width_axis]) < box_width)
                                                                      and (abs(soma_distance_array[target_neuron][k][height_axis]) < box_width))]
        #print("len(circular_radius_neurons) = " + str(len(circular_radius_neurons)))
        total_neurons_in_cluster.append(len(column_neurons))

        """
        Example of how to find neurons within x column
        """
    # evaluate if enough neurons in each cluster (if 90 percent have 10 or more)
    percentage_above_threshold = np.sum(np.array(total_neurons_in_cluster) >= suitable_number_threshold)/len(total_neurons_in_cluster)
    if percentage_above_threshold >= 0.9:
        suitable_number = True
    else:
        box_width += 50
        total_neurons_in_cluster = []

#print the suitable threshold
print("total_neurons_in_cluster Y AXIS = " + str(total_neurons_in_cluster))
print("Final box_width for circular  Y AXIS = " + str(box_width))



# column_neurons = [k for k in total_neurons_list if ((abs(soma_distance_array[target_neuron][k][width_axis]) < box_width)
#                                                                       and (abs(soma_distance_array[target_neuron][k][height_axis]) < box_width))]

# # print("len(column_neurons) = " + str(len(column_neurons)))
# # print(column_neurons)


# Find the average size of the somas:

In [ ]:
labels_neurons = pinky.ProofreadLabel & [dict(segment_id=k) for k in list(soma_cell_location_dict.keys())]
labels_neurons

In [ ]:
pinky.PymeshfixDecimatedExcitatoryStitchedMesh()

In [ ]:
#current_id = 648518346349470171
search_dict = dict(segment_id=current_id,segmentation=3)

from tqdm import tqdm
soma_bounding_box_widths_total = []

for current_id in tqdm(list(soma_cell_location_dict.keys())):
    search_dict = dict(segment_id=current_id,segmentation=3)
    #getting the face labels so can find soma submesh
    faces_labels = (labels_neurons & search_dict).fetch1("triangles")
    #print(len(faces_labels))
    soma_faces = np.where(faces_labels==5)[0]
    if len(soma_faces)>0:
        #getting the mesh to use for submesh
        verts,faces = (pinky.PymeshfixDecimatedExcitatoryStitchedMesh & search_dict).fetch1("vertices","triangles")
        #print("len(faces) = " + str(len(faces)))
        #print("len(verts) = " + str(len(verts)))

        #build the mesh and get the soma submesh:
        import trimesh
        current_neuron_mesh = trimesh.Trimesh(vertices=verts,faces=faces)
        soma_mesh = current_neuron_mesh.submesh([soma_faces],append=True)

        #soma_mesh.show()
        #get the bounding box dimensions of the soma:
        soma_bounding_box_corners = np.stack([np.min(soma_mesh.bounding_box.vertices,axis=0),
                    np.max(soma_mesh.bounding_box.vertices,axis=0)])

        soma_bounding_box_widths = soma_bounding_box_corners[1]-soma_bounding_box_corners[0]
        #print("soma_bounding_box_widths = " + str(soma_bounding_box_widths))
        soma_bounding_box_widths_total.append(soma_bounding_box_widths)

soma_bounding_box_widths_total = np.array(soma_bounding_box_widths_total)
print(soma_bounding_box_widths_total)

In [ ]:
print("Average x, y, z width for somas = " + str(np.mean(soma_bounding_box_widths_total,axis=0)))

In [ ]:
print("Final Result: " + str("Average x, y, z width for somas = [16921.20080236 16743.62922297 13732.73280046]"))

# Part 2: Start Testing 

In [ ]:
"""
Final conclusion
going to use 13000 as the width size: which is about the area of 4 somas arranged in square
--- because the width is an absolute value ---

"""

In [ ]:
interest_binned_angles,target_neuron_orientation_preferene*rad2deg

In [ ]:
# Functions that will find the subgroup:

def find_column_subgroup(width_axis ,
                         height_axis,
                        target_neuron,
                        total_neurons_list,
                        box_width=13000,
                         box_height=13000,):
    """
    Returns the list of ids for neurons in a column
    Speicifed by the parameters
    """
    column_neurons = [k for k in total_neurons_list if ((abs(soma_distance_array[target_neuron][k][width_axis]) < box_width))]
    return column_neurons

def find_radius_subgroup(target_neuron,
                        total_neurons_list,
                        circular_radius=15000):
    """
    Returns the list of ids for neurons in a column
    Speicifed by the parameters
    """
    
    circular_radius_neurons = [k for k in total_neurons_list if soma_distance_array[target_neuron][k]["distance"] < circular_radius ]
    return circular_radius_neurons
                                                    

In [ ]:
"""
Shows whole process of where start with a 

"""

#
target_neuron = 648518346341371119
target_neuron_orientation_preferene = seg_id_to_orientation[target_neuron]

"""
Example of how to find neurons within a certain radius
"""
total_neurons_list = list(soma_cell_location_dict.keys())
total_neurons_list.remove(target_neuron)

circular_radius = 15000 #test 1
circular_radius_neurons = [k for k in total_neurons_list if soma_distance_array[target_neuron][k]["distance"] < circular_radius ]
print("len(circular_radius_neurons) = " + str(len(circular_radius_neurons)))

"""
Example of how to find neurons within x column
"""

box_width = 13000
box_height = 13000
width_axis = "x_distance"
height_axis = "z_distance"

column_neurons = [k for k in total_neurons_list if ((abs(soma_distance_array[target_neuron][k][width_axis]) < box_width)
                                                                      and (abs(soma_distance_array[target_neuron][k][height_axis]) < box_height))]

print("len(column_neurons) = " + str(len(column_neurons)))
print(column_neurons)

"""
Get the orientation distribution and difference in orientation distributions for a target neuron and it's restricted neighbors

This example shows the circular_radius_neurons
"""
global_neuron_group = total_neurons_list = list(soma_cell_location_dict.keys())
print("global_neuron_group = " + str(len(global_neuron_group)))
restricted_neuron_group = circular_radius_neurons
print("restricted_neuron_group = " + str(len(restricted_neuron_group)))

#get the orientations of these groups
restricted_neuron_group_orientations = [seg_id_to_orientation[k] for k in restricted_neuron_group] #NOT INCLUDE TARGET NEURON

global_neuron_group_orientations = [seg_id_to_orientation[k] for k in global_neuron_group] # DOES INCLUDE TARGET NEURON


current_n_bins = 20
current_lower_bin_bound = 0
current_upper_bin_bound = np.pi

binned_angles,binned_angles_histogram = bin_orientation_data(restricted_neuron_group_orientations,
                                                         n_bins = current_n_bins,
                                                        lower_bin_bound = current_lower_bin_bound,
                                                         upper_bin_bound = current_upper_bin_bound)

global_binned_angles,global_binned_angles_histogram = bin_orientation_data(global_neuron_group_orientations,
                                                         n_bins = current_n_bins,
                                                        lower_bin_bound = current_lower_bin_bound,
                                                         upper_bin_bound = current_upper_bin_bound)

interest_binned_angles,interest_binned_angles_histogram = bin_orientation_data([target_neuron_orientation_preferene],
                                                        n_bins = current_n_bins,
                                                        lower_bin_bound = current_lower_bin_bound,
                                                         upper_bin_bound = current_upper_bin_bound)

#print(interest_binned_angles)

graph_binned_orientation_data(binned_angles,
                                binned_angles_histogram,
                                  neuron_id=target_neuron,
                                  title="y_column_"+str(box_width),
                              n_bins=current_n_bins,
                              global_binned_angles = global_binned_angles,
                                global_binned_angles_histogram= global_binned_angles_histogram,
                                 graphs_to_plot=["local","global"],
                                 figure_size = (20,20),
                                 circular_flag = True,
                                 target_neuron_orientation = interest_binned_angles)




In [ ]:
#original volume dimensions
x_box = 250000
y_box = 140000
z_box = 90000


# test 2 (more strict to try and keep around 10 neurons for at least 90 %)
y_width = y_height = 6600
z_width = z_height = 11900
x_width = x_height = 2850
circular_radius = 13700

# calculate the total volume for certain parameters
y_col_volume = (2*y_width) * (2*y_height) * y_box
x_col_volume = (2*x_width) * (2*x_height) * x_box
z_col_volume = (2*z_width) * (2*z_height) * z_box
sphere_volume = 4.0/3.0*np.pi*(circular_radius**3)

print(f"y_col_volume = {y_col_volume} \
\nx_col_volume = {x_col_volume}\
\nz_col_volume = {z_col_volume}\
\nsphere_volume = {sphere_volume}")


print("\n\nThe relative scaling")
print(f"y_col_volume = {y_col_volume/sphere_volume} \
\nx_col_volume = {x_col_volume/sphere_volume}\
\nz_col_volume = {z_col_volume/sphere_volume}\
\nsphere_volume = {sphere_volume/sphere_volume}")

# find scaling factors to make everyone the same volume
def get_extra_width(final_volume,current_height,current_width):
    extra_width = np.sqrt((final_volume)/current_height)/2 - current_width
    return extra_width
#     if extra_width < 0:
#         return - np.sqrt(np.abs(extra_width))
#     else:
#         return np.sqrt(np.abs(extra_width))
    

y_extra_width = get_extra_width(sphere_volume,y_box,y_width)
#check that the height will now be the same
y_new_width = y_width + y_extra_width


x_extra_width = get_extra_width(sphere_volume,x_box,x_width)
#check that the height will now be the same
x_new_width = x_width + x_extra_width

z_extra_width = get_extra_width(sphere_volume,z_box,z_width)
#check that the height will now be the same
z_new_width = z_width + z_extra_width

"""
4*w*h*box + 8*extra*h*ybox + 4*extra^2*y_box = new_volume
8*extra*width + 4*extra**2 = (new_volum - current_volume)/y_box

"""

print("\n New volume scales")
y_col_volume_new = (2*y_new_width) * (2*y_new_width) * y_box
z_col_volume_new = (2*z_new_width) * (2*z_new_width) * z_box
x_col_volume_new = (2*x_new_width) * (2*x_new_width) * x_box
print(y_col_volume_new/sphere_volume,z_col_volume_new/sphere_volume,x_col_volume_new/sphere_volume)

print("\n New widths for exact same volume")
print("y_new_width = " + str(y_new_width))
print("z_new_width = " + str(z_new_width))
print("x_new_width = " + str(x_new_width))

# How could we add different scaling to make sure holds true
scale_factor = 2
circular_radius_scaled = (scale_factor**(1/3))*circular_radius
y_new_width_scaled = (scale_factor**(1/2))*y_new_width
z_new_width_scaled = (scale_factor**(1/2))*z_new_width
x_new_width_scaled = (scale_factor**(1/2))*x_new_width

y_col_volume_scaled = (2*y_new_width_scaled) * (2*y_new_width_scaled) * y_box
z_col_volume_scaled = (2*z_new_width_scaled) * (2*z_new_width_scaled) * z_box
x_col_volume_scaled = (2*x_new_width_scaled) * (2*x_new_width_scaled) * x_box
sphere_volume_scaled = (4.0/3.0)*np.pi*(circular_radius_scaled**3)

print("\n New widths and radius SCALED by factor of " + str(scale_factor))
print("y_new_width_scaled = " + str(y_new_width_scaled))
print("z_new_width_scaled = " + str(z_new_width_scaled))
print("x_new_width_scaled = " + str(x_new_width_scaled))
print("circular_radius_scaled = " + str(circular_radius_scaled))

print(y_col_volume_scaled/sphere_volume_scaled,z_col_volume_scaled/sphere_volume_scaled,x_col_volume_scaled/sphere_volume_scaled)

scale_factor = 0.5
circular_radius_scaled = (scale_factor**(1/3))*circular_radius
y_new_width_scaled = (scale_factor**(1/2))*y_new_width
z_new_width_scaled = (scale_factor**(1/2))*z_new_width
x_new_width_scaled = (scale_factor**(1/2))*x_new_width

print("\n New widths and radius SCALED by factor of " + str(scale_factor))
print("y_new_width_scaled = " + str(y_new_width_scaled))
print("z_new_width_scaled = " + str(z_new_width_scaled))
print("x_new_width_scaled = " + str(x_new_width_scaled))
print("circular_radius_scaled = " + str(circular_radius_scaled))


In [ ]:
"""Steps for analysis: For each neuron
    1) Get the subgroup (from columns or spheres), can adjust for the same number of neurons or same distance
    2) Calculate the average mse of the orientations of the subgroup from the neuron of interest (aka get the variance)
    3) Calculate the same thing as step 2 but with a random shuffle of neurons from outside the volume (repeat for lots of shuffles so get the distribution)
    4) Compare Step 2 and compute the probability of getting that from the distribution gotten in step 3
    Repeat 1- 4 for all neurons

Look at what the average probability is for all neurons and see if there is instane wher
    
    
mse from the columns

keep the column heights and sphere volume the same"""
import time
def calc_mse(restricted_neuron_group,
             target_scaled
            ):
    
    restricted_neuron_group_orientation_preference = np.array([seg_id_to_orientation[k] for k in restricted_neuron_group])

    #now to get the total list shifted by subtracting off the target neuron
    total_neurons_scaled = restricted_neuron_group_orientation_preference*2

    differences = pycs.pairwise_cdiff(total_neurons_scaled,target_scaled)/2
    squared_difference = np.mean(differences**2)
    
    return squared_difference

mse_data = dict()
#target_neuron = 648518346341371119
start_time = time.time()
for target_neuron in soma_cell_location_dict.keys():
    

    mse_data[target_neuron] = dict()

    target_neuron_orientation_preferene = seg_id_to_orientation[target_neuron]


    target_scaled = target_neuron_orientation_preferene*2
    """
    Example of how to find neurons within a certain radius
    """
    total_neurons_list = list(soma_cell_location_dict.keys())
    total_neurons_list.remove(target_neuron)

    """
    Example of how to find neurons within x column
    """

    # test 1
    y_width = y_height = 11000
    z_width = z_height = 11000
    x_width = x_height = 11000
    circular_radius = 13000
    
    # test 2 (more strict to try and keep around 10 neurons for at least 90 %)
    y_width = y_height = 6600
    z_width = z_height = 11900
    x_width = x_height = 2850
    circular_radius = 13700
    
    #test 3 (with equal volume) 
    y_new_width = 4385.621445508057
    z_new_width = 5469.83095905981
    x_new_width = 3281.8985754358864
    
    y_width = y_height = y_new_width
    z_width = z_height = z_new_width
    x_width = x_height = x_new_width
    
    #test 4 (scaled by 2 with equal volume)
    y_new_width_scaled = 6202.2053276717925
    z_new_width_scaled = 7735.509126190617
    x_new_width_scaled = 4641.305475714371
    circular_radius_scaled = 17260.91838355976
    
    y_width = y_height = y_new_width_scaled
    z_width = z_height = z_new_width_scaled
    x_width = x_height = x_new_width_scaled
    circular_radius = circular_radius_scaled
    
    # test 5
    #New widths and radius SCALED by factor of 0.5
    y_new_width_scaled = 3101.1026638358962
    z_new_width_scaled = 3867.7545630953086
    x_new_width_scaled = 2320.6527378571855
    circular_radius_scaled = 10873.697205982167
    
    y_width = y_height = y_new_width_scaled
    z_width = z_height = z_new_width_scaled
    x_width = x_height = x_new_width_scaled
    circular_radius = circular_radius_scaled
    
    width_axis = "x_distance"; height_axis = "z_distance"
    y_column_subgroup = find_column_subgroup(width_axis,height_axis,target_neuron,total_neurons_list,y_width,y_height)

    
    width_axis = "x_distance"; height_axis = "y_distance"
    z_column_subgroup = find_column_subgroup(width_axis,height_axis,target_neuron,total_neurons_list,z_width,z_height)

    
    width_axis = "y_distance"; height_axis = "z_distance"
    x_column_subgroup = find_column_subgroup(width_axis,height_axis,target_neuron,total_neurons_list,x_width,x_height)

    
    circular_subgroup = find_radius_subgroup(target_neuron,total_neurons_list,circular_radius)


    subgroups_list = dict(y_column_subgroup=y_column_subgroup,
                         z_column_subgroup=z_column_subgroup,
                         x_column_subgroup=x_column_subgroup,
                         circular_subgroup=circular_subgroup)

    n_random_shuffles = 1000


    
    for subgroup_name,restricted_neuron_group in subgroups_list.items():
        #get the mse of that subgroup
        restricted_neuron_group_mse = calc_mse(restricted_neuron_group,
                                              target_scaled)
        outside_neurons = [k for k in total_neurons_list if 
                       ((k not in restricted_neuron_group) and 
                        k != target_neuron)]
#         print("len(restricted_neuron_group) = " + str(len(restricted_neuron_group)))
#         print("len(outside_neurons) = " + str(len(outside_neurons)))
        
        random_shuffles_mse = []
        for i in range(0,n_random_shuffles):
            #get a random list from outside the subgroup
            if len(restricted_neuron_group) < len(outside_neurons):
                outside_neurons_rand = random.sample(outside_neurons,len(restricted_neuron_group))
            else:
                if i == 1:
                    print("not using sampling")
                outside_neurons_rand = outside_neurons
            
            if len(restricted_neuron_group) < len(outside_neurons_rand):
                print("RESTRICTED GROUP NUMBER IS LESS")
            random_shuffles_mse.append(calc_mse(outside_neurons_rand,
                                              target_scaled))

        #save the random shuffles
        mse_data[target_neuron][subgroup_name] = dict() 
        mse_data[target_neuron][subgroup_name]["shuffles"] = random_shuffles_mse
        mse_data[target_neuron][subgroup_name]["real_group"] = restricted_neuron_group_mse
        mse_data[target_neuron][subgroup_name]["real_group_len"] = len(restricted_neuron_group)
        
        probability_less_than = np.sum(random_shuffles_mse<restricted_neuron_group_mse)/len(random_shuffles_mse)
        mse_data[target_neuron][subgroup_name]["probability"] = probability_less_than
print(f"Total time for neurons = " + str(time.time() - start_time))

In [ ]:

x_parameter_string = f"x_width = {x_width}, x_height = {x_height}"
y_parameter_string = f"y_width = {y_width}, y_height = {y_height}"
z_parameter_string = f"z_width = {z_width}, z_height = {z_height}"
sphere_parameter_string = f"circular_radius = {circular_radius}"


whole_parameters_string = "\n"+x_parameter_string+ \
                            "\n"+y_parameter_string+ \
                            "\n"+z_parameter_string+ \
                            "\n"+sphere_parameter_string

parameter_strings = [y_parameter_string,z_parameter_string,x_parameter_string,
                    sphere_parameter_string]

whole_parameters_string

#get all of the probabilities for each group

column_sphere_names = subgroups_list.keys()
total_probabilities = []
total_group_number = []
column_sphere_subgroup_number = dict()
column_sphere_probabilities = dict()
for volume_name,param in zip(column_sphere_names,parameter_strings):
    column_sphere_probabilities[volume_name] = []
    column_sphere_subgroup_number[volume_name] = []
    for a in mse_data.keys():
            column_sphere_probabilities[volume_name].append(mse_data[a][volume_name]["probability"])
            column_sphere_subgroup_number[volume_name].append(mse_data[a][volume_name]["real_group_len"])
    plt.figure()
    plt.hist(column_sphere_probabilities[volume_name],bins=100)
    mean_group_number = np.mean(column_sphere_subgroup_number[volume_name])
    std_dev_group_number = np.std(column_sphere_subgroup_number[volume_name])
    plt.title(volume_name  + " probability distributions\n" + param 
                           + "\nmean subgroup number = " + str(np.round(mean_group_number,3))
                           + "\nstd dev subgroup number = " + str(np.round(std_dev_group_number,3))
                            + "\n Mean probability = " + str(np.round(np.mean(column_sphere_probabilities[volume_name]),3)))
    total_group_number += column_sphere_subgroup_number[volume_name]
    total_probabilities += column_sphere_probabilities[volume_name]
    
    
mean_group_number = np.mean(total_group_number)
std_dev_group_number = np.std(total_group_number)

plt.figure()
plt.hist(total_probabilities,bins=100)
plt.title("Total probability distributions" + whole_parameters_string
         + "\nmean subgroup number = " + str(np.round(mean_group_number,3))
         + "\nstd dev subgroup number = " + str(np.round(std_dev_group_number,3))
         + "\n Mean probability = " + str(np.round(np.mean(total_probabilities),3)))

# Do the same probability analysis for only neurons with orientations with certain preference

In [ ]:
mse_radian_subgroup = dict()
window_size_degrees = 180/16/2
window_size_radians = window_size_degrees/rad2deg*4

#how the radians will move when graphing them
n_window_samples = 50
window_radians_stepsize = np.pi/n_window_samples

total_center_radians = np.linspace(0,np.pi,n_window_samples)

In [ ]:



#-------------------starting loop through all middle orientations-----------------------#




for middle_orientation in tqdm(total_center_radians):
    mse_radian_subgroup[middle_orientation] = dict()

    #print(f"Radians range = {(middle_orientation - window_size_radians,middle_orientation + window_size_radians)}")

    #get the orientation differences of all
    target_neurons_to_iterate_through = [k for k in soma_cell_location_dict.keys() if 
                                         np.abs(pycs.cdiff(seg_id_to_orientation[k]*2,middle_orientation*2)/2) < window_size_radians]

    #print("len(target_neurons_to_iterate_through) = " + str(len(target_neurons_to_iterate_through)))
    mse_radian_subgroup[middle_orientation]["neuron_number"] = len(target_neurons_to_iterate_through)

    #checking that correctly getting the subgroup
    orientations_neurons_to_iterate_through = [seg_id_to_orientation[k] for k in target_neurons_to_iterate_through]
    x = plt.hist(orientations_neurons_to_iterate_through,bins=20)
    plt.xlim([0,np.pi])
    print((min(x[1]),max(x[1])))


    plotting = False

    mse_data = dict()
    #target_neuron = 648518346341371119
    start_time = time.time()

    for target_neuron in target_neurons_to_iterate_through:


        mse_data[target_neuron] = dict()

        target_neuron_orientation_preferene = seg_id_to_orientation[target_neuron]


        target_scaled = target_neuron_orientation_preferene*2
        """
        Example of how to find neurons within a certain radius
        """
        total_neurons_list = list(soma_cell_location_dict.keys())
        total_neurons_list.remove(target_neuron)

        """
        Example of how to find neurons within x column
        """

        # test 1
        y_width = y_height = 11000
        z_width = z_height = 11000
        x_width = x_height = 11000
        circular_radius = 13000

        # test 2 (more strict to try and keep around 10 neurons for at least 90 %)
        y_width = y_height = 6600
        z_width = z_height = 11900
        x_width = x_height = 2850
        circular_radius = 13700

        #test 3 (with equal volume) 
        y_new_width = 4385.621445508057
        z_new_width = 5469.83095905981
        x_new_width = 3281.8985754358864

        y_width = y_height = y_new_width
        z_width = z_height = z_new_width
        x_width = x_height = x_new_width

    #     #test 4 (scaled by 2 with equal volume)
    #     y_new_width_scaled = 6202.2053276717925
    #     z_new_width_scaled = 7735.509126190617
    #     x_new_width_scaled = 4641.305475714371
    #     circular_radius_scaled = 17260.91838355976

    #     y_width = y_height = y_new_width_scaled
    #     z_width = z_height = z_new_width_scaled
    #     x_width = x_height = x_new_width_scaled
    #     circular_radius = circular_radius_scaled

    #     # test 5
    #     #New widths and radius SCALED by factor of 0.5
    #     y_new_width_scaled = 3101.1026638358962
    #     z_new_width_scaled = 3867.7545630953086
    #     x_new_width_scaled = 2320.6527378571855
    #     circular_radius_scaled = 10873.697205982167

    #     y_width = y_height = y_new_width_scaled
    #     z_width = z_height = z_new_width_scaled
    #     x_width = x_height = x_new_width_scaled
    #     circular_radius = circular_radius_scaled

        width_axis = "x_distance"; height_axis = "z_distance"
        y_column_subgroup = find_column_subgroup(width_axis,height_axis,target_neuron,total_neurons_list,y_width,y_height)


        width_axis = "x_distance"; height_axis = "y_distance"
        z_column_subgroup = find_column_subgroup(width_axis,height_axis,target_neuron,total_neurons_list,z_width,z_height)


        width_axis = "y_distance"; height_axis = "z_distance"
        x_column_subgroup = find_column_subgroup(width_axis,height_axis,target_neuron,total_neurons_list,x_width,x_height)


        circular_subgroup = find_radius_subgroup(target_neuron,total_neurons_list,circular_radius)


        subgroups_list = dict(y_column_subgroup=y_column_subgroup,
                             z_column_subgroup=z_column_subgroup,
                             x_column_subgroup=x_column_subgroup,
                             circular_subgroup=circular_subgroup)

        n_random_shuffles = 1000



        for subgroup_name,restricted_neuron_group in subgroups_list.items():
            #get the mse of that subgroup
            restricted_neuron_group_mse = calc_mse(restricted_neuron_group,
                                                  target_scaled)
            outside_neurons = [k for k in total_neurons_list if 
                           ((k not in restricted_neuron_group) and 
                            k != target_neuron)]
    #         print("len(restricted_neuron_group) = " + str(len(restricted_neuron_group)))
    #         print("len(outside_neurons) = " + str(len(outside_neurons)))

            random_shuffles_mse = []
            for i in range(0,n_random_shuffles):
                #get a random list from outside the subgroup
                if len(restricted_neuron_group) < len(outside_neurons):
                    outside_neurons_rand = random.sample(outside_neurons,len(restricted_neuron_group))
                else:
                    if i == 1:
                        print("not using sampling")
                    outside_neurons_rand = outside_neurons

                if len(restricted_neuron_group) < len(outside_neurons_rand):
                    print("RESTRICTED GROUP NUMBER IS LESS")
                random_shuffles_mse.append(calc_mse(outside_neurons_rand,
                                                  target_scaled))

            #save the random shuffles
            mse_data[target_neuron][subgroup_name] = dict() 
            mse_data[target_neuron][subgroup_name]["shuffles"] = random_shuffles_mse
            mse_data[target_neuron][subgroup_name]["real_group"] = restricted_neuron_group_mse
            mse_data[target_neuron][subgroup_name]["real_group_len"] = len(restricted_neuron_group)

            probability_less_than = np.sum(random_shuffles_mse<restricted_neuron_group_mse)/len(random_shuffles_mse)
            mse_data[target_neuron][subgroup_name]["probability"] = probability_less_than
    print(f"Total time for neurons = " + str(time.time() - start_time))


    x_parameter_string = f"x_width = {x_width}, x_height = {x_height}"
    y_parameter_string = f"y_width = {y_width}, y_height = {y_height}"
    z_parameter_string = f"z_width = {z_width}, z_height = {z_height}"
    sphere_parameter_string = f"circular_radius = {circular_radius}"


    whole_parameters_string = "\n"+x_parameter_string+ \
                                "\n"+y_parameter_string+ \
                                "\n"+z_parameter_string+ \
                                "\n"+sphere_parameter_string

    parameter_strings = [y_parameter_string,z_parameter_string,x_parameter_string,
                        sphere_parameter_string]

    whole_parameters_string

    #get all of the probabilities for each group

    column_sphere_names = subgroups_list.keys()
    total_probabilities = []
    total_group_number = []
    column_sphere_subgroup_number = dict()
    column_sphere_probabilities = dict()
    for volume_name,param in zip(column_sphere_names,parameter_strings):
        column_sphere_probabilities[volume_name] = []
        column_sphere_subgroup_number[volume_name] = []
        for a in mse_data.keys():
                column_sphere_probabilities[volume_name].append(mse_data[a][volume_name]["probability"])
                column_sphere_subgroup_number[volume_name].append(mse_data[a][volume_name]["real_group_len"])
        mean_group_number = np.mean(column_sphere_subgroup_number[volume_name])
        std_dev_group_number = np.std(column_sphere_subgroup_number[volume_name])
        mean_probability = np.mean(column_sphere_probabilities[volume_name])
        if plotting:
            plt.figure()
            plt.hist(column_sphere_probabilities[volume_name],bins=100)

            plt.title(volume_name  + " probability distributions\n" + param 
                                   + "\nmean subgroup number = " + str(np.round(mean_group_number,3))
                                   + "\nstd dev subgroup number = " + str(np.round(std_dev_group_number,3))
                                    + "\n Mean probability = " + str(np.round(mean_probability,3)))

        #store the mean probability, subgroup number and std deviation in the largest dictionary so 
        # can compare as iterate over the radians
        mse_radian_subgroup[middle_orientation][volume_name] = dict(mean_group_number=mean_group_number,
                                                                   std_dev_group_number=std_dev_group_number,
                                                                   mean_probability=mean_probability)


        total_group_number += column_sphere_subgroup_number[volume_name]
        total_probabilities += column_sphere_probabilities[volume_name]


    mean_group_number = np.mean(total_group_number)
    std_dev_group_number = np.std(total_group_number)
    mean_probability = np.mean(total_probabilities)

    mse_radian_subgroup[middle_orientation]["total"] = dict(mean_group_number=mean_group_number,
                                                                   std_dev_group_number=std_dev_group_number,
                                                                   mean_probability=mean_probability)
    if plotting:
        plt.figure()
        plt.hist(total_probabilities,bins=100)
        plt.title("Total probability distributions" + whole_parameters_string
                 + "\nmean subgroup number = " + str(np.round(mean_group_number,3))
                 + "\nstd dev subgroup number = " + str(np.round(std_dev_group_number,3))
                 + "\n Mean probability = " + str(np.round(mean_probability,3)))

In [ ]:
"""
Graphs want to do 
1) The mean probability of each group vs. center
2) The mean group number of each group vs. center
3) The std dev group number of each group vs. center
4) Number neurons that were tested in each radian group vs. center

"""

x = list(mse_radian_subgroup.keys())
column_type = list(mse_radian_subgroup[list(x)[0]].keys())
if "neuron_number" in column_type:
    column_type.remove("neuron_number")

#get the data for each column type
probability_data = dict()
std_dev_group_data = dict()
mean_group_data = dict()
for t in column_type:
    probability_data[t] = [v[t]["mean_probability"] for k,v in mse_radian_subgroup.items()]
    std_dev_group_data[t] = [v[t]["mean_group_number"] for k,v in mse_radian_subgroup.items()]
    mean_group_data[t] = [v[t]["std_dev_group_number"] for k,v in mse_radian_subgroup.items()]
    
#get the neuron number for each radian group
neuron_number_data = [mse_radian_subgroup[k]["neuron_number"] for k in mse_radian_subgroup.keys()]

window_size_radians_rounded = np.round(window_size_radians,3)
plt.figure()
plt.title(" Mean Probabilty vs. Radian center with window size +/- " + str(window_size_radians_rounded) + " radians")
for t,y in probability_data.items():
    plt.plot(x,y,label=t)
plt.legend()
ax = plt.gca()
box = ax.get_position()
ax.set_position([box.x0,box.y0,box.width*0.8,box.height])
ax.legend(loc='center left',bbox_to_anchor=(1,0.5))

fig = plt.figure()

plt.title(" Mean group number vs. Radian center with window size +/- " + str(window_size_radians_rounded) + " radians")
for t,y in mean_group_data.items():
    plt.plot(x,y,label=t)
plt.legend()
ax = plt.gca()
box = ax.get_position()
ax.set_position([box.x0,box.y0,box.width*0.8,box.height])
ax.legend(loc='center left',bbox_to_anchor=(1,0.5))

plt.figure()
plt.title(" Std dev group number vs. Radian center with window size +/- " + str(window_size_radians_rounded) + " radians")
for t,y in std_dev_group_data.items():
    plt.plot(x,y,label=t)
plt.legend()
ax = plt.gca()
box = ax.get_position()
ax.set_position([box.x0,box.y0,box.width*0.8,box.height])
ax.legend(loc='center left',bbox_to_anchor=(1,0.5))

plt.figure()
plt.title(" n_neurons in radian range vs. Radian center with window size +/- " + str(window_size_radians_rounded) + " radians")
plt.plot(x,neuron_number_data)

In [ ]:
# Iterating over differing the scale and seeing how that changes:
mse_scale_subgroup = dict()
n_window_samples = 20
total_scalings = np.linspace(0.25,4,n_window_samples)

In [ ]:



#-------------------starting loop through all middle orientations-----------------------#
for scale_facotr in tqdm(total_scalings):
    mse_scale_subgroup[scale_facotr] = dict()


    plotting = False

    mse_data = dict()
    #target_neuron = 648518346341371119
    start_time = time.time()

    for target_neuron in soma_cell_location_dict.keys():


        mse_data[target_neuron] = dict()

        target_neuron_orientation_preferene = seg_id_to_orientation[target_neuron]


        target_scaled = target_neuron_orientation_preferene*2
        """
        Example of how to find neurons within a certain radius
        """
        total_neurons_list = list(soma_cell_location_dict.keys())
        total_neurons_list.remove(target_neuron)

        """
        Example of how to find neurons within x column
        """

        # test 1
        y_width = y_height = 11000
        z_width = z_height = 11000
        x_width = x_height = 11000
        circular_radius = 13000

        # test 2 (more strict to try and keep around 10 neurons for at least 90 %)
        y_width = y_height = 6600
        z_width = z_height = 11900
        x_width = x_height = 2850
        circular_radius = 13700

        #test 3 (with equal volume) 
        y_new_width = 4385.621445508057
        z_new_width = 5469.83095905981
        x_new_width = 3281.8985754358864

        y_width = y_height = y_new_width
        z_width = z_height = z_new_width
        x_width = x_height = x_new_width

        
        circular_radius_scaled = (scale_factor**(1/3))*circular_radius
        y_new_width_scaled = (scale_factor**(1/2))*y_new_width
        z_new_width_scaled = (scale_factor**(1/2))*z_new_width
        x_new_width_scaled = (scale_factor**(1/2))*x_new_width

#         print("\n New widths and radius SCALED by factor of " + str(scale_factor))
#         print("y_new_width_scaled = " + str(y_new_width_scaled))
#         print("z_new_width_scaled = " + str(z_new_width_scaled))
#         print("x_new_width_scaled = " + str(x_new_width_scaled))
#         print("circular_radius_scaled = " + str(circular_radius_scaled))
        
        y_width = y_height = y_new_width_scaled
        z_width = z_height = z_new_width_scaled
        x_width = x_height = x_new_width_scaled
        circular_radius = circular_radius_scaled
        

        width_axis = "x_distance"; height_axis = "z_distance"
        y_column_subgroup = find_column_subgroup(width_axis,height_axis,target_neuron,total_neurons_list,y_width,y_height)


        width_axis = "x_distance"; height_axis = "y_distance"
        z_column_subgroup = find_column_subgroup(width_axis,height_axis,target_neuron,total_neurons_list,z_width,z_height)


        width_axis = "y_distance"; height_axis = "z_distance"
        x_column_subgroup = find_column_subgroup(width_axis,height_axis,target_neuron,total_neurons_list,x_width,x_height)


        circular_subgroup = find_radius_subgroup(target_neuron,total_neurons_list,circular_radius)


        subgroups_list = dict(y_column_subgroup=y_column_subgroup,
                             z_column_subgroup=z_column_subgroup,
                             x_column_subgroup=x_column_subgroup,
                             circular_subgroup=circular_subgroup)

        n_random_shuffles = 1000



        for subgroup_name,restricted_neuron_group in subgroups_list.items():
            #get the mse of that subgroup
            restricted_neuron_group_mse = calc_mse(restricted_neuron_group,
                                                  target_scaled)
            outside_neurons = [k for k in total_neurons_list if 
                           ((k not in restricted_neuron_group) and 
                            k != target_neuron)]
    #         print("len(restricted_neuron_group) = " + str(len(restricted_neuron_group)))
    #         print("len(outside_neurons) = " + str(len(outside_neurons)))

            random_shuffles_mse = []
            for i in range(0,n_random_shuffles):
                #get a random list from outside the subgroup
                if len(restricted_neuron_group) < len(outside_neurons):
                    outside_neurons_rand = random.sample(outside_neurons,len(restricted_neuron_group))
                else:
                    if i == 1:
                        print("not using sampling")
                    outside_neurons_rand = outside_neurons

                if len(restricted_neuron_group) < len(outside_neurons_rand):
                    print("RESTRICTED GROUP NUMBER IS LESS")
                random_shuffles_mse.append(calc_mse(outside_neurons_rand,
                                                  target_scaled))

            #save the random shuffles
            mse_data[target_neuron][subgroup_name] = dict() 
            mse_data[target_neuron][subgroup_name]["shuffles"] = random_shuffles_mse
            mse_data[target_neuron][subgroup_name]["real_group"] = restricted_neuron_group_mse
            mse_data[target_neuron][subgroup_name]["real_group_len"] = len(restricted_neuron_group)

            probability_less_than = np.sum(random_shuffles_mse<restricted_neuron_group_mse)/len(random_shuffles_mse)
            mse_data[target_neuron][subgroup_name]["probability"] = probability_less_than
    print(f"Total time for neurons = " + str(time.time() - start_time))


    x_parameter_string = f"x_width = {x_width}, x_height = {x_height}"
    y_parameter_string = f"y_width = {y_width}, y_height = {y_height}"
    z_parameter_string = f"z_width = {z_width}, z_height = {z_height}"
    sphere_parameter_string = f"circular_radius = {circular_radius}"


    whole_parameters_string = "\n"+x_parameter_string+ \
                                "\n"+y_parameter_string+ \
                                "\n"+z_parameter_string+ \
                                "\n"+sphere_parameter_string

    parameter_strings = [y_parameter_string,z_parameter_string,x_parameter_string,
                        sphere_parameter_string]

    whole_parameters_string

    #get all of the probabilities for each group

    column_sphere_names = subgroups_list.keys()
    total_probabilities = []
    total_group_number = []
    column_sphere_subgroup_number = dict()
    column_sphere_probabilities = dict()
    for volume_name,param in zip(column_sphere_names,parameter_strings):
        column_sphere_probabilities[volume_name] = []
        column_sphere_subgroup_number[volume_name] = []
        for a in mse_data.keys():
                column_sphere_probabilities[volume_name].append(mse_data[a][volume_name]["probability"])
                column_sphere_subgroup_number[volume_name].append(mse_data[a][volume_name]["real_group_len"])
        mean_group_number = np.mean(column_sphere_subgroup_number[volume_name])
        std_dev_group_number = np.std(column_sphere_subgroup_number[volume_name])
        mean_probability = np.mean(column_sphere_probabilities[volume_name])
        if plotting:
            plt.figure()
            plt.hist(column_sphere_probabilities[volume_name],bins=100)

            plt.title(volume_name  + " probability distributions\n" + param 
                                   + "\nmean subgroup number = " + str(np.round(mean_group_number,3))
                                   + "\nstd dev subgroup number = " + str(np.round(std_dev_group_number,3))
                                    + "\n Mean probability = " + str(np.round(mean_probability,3)))

        #store the mean probability, subgroup number and std deviation in the largest dictionary so 
        # can compare as iterate over the radians
        mse_scale_subgroup[scale_facotr][volume_name] = dict(mean_group_number=mean_group_number,
                                                                   std_dev_group_number=std_dev_group_number,
                                                                   mean_probability=mean_probability)


        total_group_number += column_sphere_subgroup_number[volume_name]
        total_probabilities += column_sphere_probabilities[volume_name]


    mean_group_number = np.mean(total_group_number)
    std_dev_group_number = np.std(total_group_number)
    mean_probability = np.mean(total_probabilities)

    mse_scale_subgroup[scale_facotr]["total"] = dict(mean_group_number=mean_group_number,
                                                                   std_dev_group_number=std_dev_group_number,
                                                                   mean_probability=mean_probability)
    if plotting:
        plt.figure()
        plt.hist(total_probabilities,bins=100)
        plt.title("Total probability distributions" + whole_parameters_string
                 + "\nmean subgroup number = " + str(np.round(mean_group_number,3))
                 + "\nstd dev subgroup number = " + str(np.round(std_dev_group_number,3))
                 + "\n Mean probability = " + str(np.round(mean_probability,3)))

In [ ]:
mse_scale_subgroup.keys()

In [ ]:
"""
Graphs want to do 
1) The mean probability of each group vs. center
2) The mean group number of each group vs. center
3) The std dev group number of each group vs. center
4) Number neurons that were tested in each radian group vs. center

"""

x = list(mse_scale_subgroup.keys())
column_type = list(mse_scale_subgroup[list(x)[0]].keys())

#get the data for each column type
probability_data = dict()
std_dev_group_data = dict()
mean_group_data = dict()
for t in column_type:
    probability_data[t] = [v[t]["mean_probability"] for k,v in mse_scale_subgroup.items()]
    std_dev_group_data[t] = [v[t]["mean_group_number"] for k,v in mse_scale_subgroup.items()]
    mean_group_data[t] = [v[t]["std_dev_group_number"] for k,v in mse_scale_subgroup.items()]
    

plt.figure()
plt.title(" Mean Probabilty vs. Scale")
for t,y in probability_data.items():
    plt.plot(x,y,label=t)
plt.legend()
ax = plt.gca()
box = ax.get_position()
ax.set_position([box.x0,box.y0,box.width*0.8,box.height])
ax.legend(loc='center left',bbox_to_anchor=(1,0.5))

fig = plt.figure()

plt.title(" Mean group number vs. Scale")
for t,y in mean_group_data.items():
    plt.plot(x,y,label=t)
plt.legend()
ax = plt.gca()
box = ax.get_position()
ax.set_position([box.x0,box.y0,box.width*0.8,box.height])
ax.legend(loc='center left',bbox_to_anchor=(1,0.5))

plt.figure()
plt.title(" Std dev group number vs. Scale")
for t,y in std_dev_group_data.items():
    plt.plot(x,y,label=t)
plt.legend()
ax = plt.gca()
box = ax.get_position()
ax.set_position([box.x0,box.y0,box.width*0.8,box.height])
ax.legend(loc='center left',bbox_to_anchor=(1,0.5))

# Heat map of the highest probabilites: 

In [ ]:
"""Steps for analysis: For each neuron
    1) Get the subgroup (from columns or spheres), can adjust for the same number of neurons or same distance
    2) Calculate the average mse of the orientations of the subgroup from the neuron of interest (aka get the variance)
    3) Calculate the same thing as step 2 but with a random shuffle of neurons from outside the volume (repeat for lots of shuffles so get the distribution)
    4) Compare Step 2 and compute the probability of getting that from the distribution gotten in step 3
    Repeat 1- 4 for all neurons

Look at what the average probability is for all neurons and see if there is instane wher
    
    
mse from the columns

keep the column heights and sphere volume the same"""
import time
def calc_mse(restricted_neuron_group,
             target_scaled
            ):
    
    restricted_neuron_group_orientation_preference = np.array([seg_id_to_orientation[k] for k in restricted_neuron_group])

    #now to get the total list shifted by subtracting off the target neuron
    total_neurons_scaled = restricted_neuron_group_orientation_preference*2

    differences = pycs.pairwise_cdiff(total_neurons_scaled,target_scaled)/2
    squared_difference = np.mean(differences**2)
    
    return squared_difference

mse_data = dict()

start_time = time.time()
for target_neuron in soma_cell_location_dict.keys():
    

    mse_data[target_neuron] = dict()

    target_neuron_orientation_preferene = seg_id_to_orientation[target_neuron]


    target_scaled = target_neuron_orientation_preferene*2
    """
    Example of how to find neurons within a certain radius
    """
    total_neurons_list = list(soma_cell_location_dict.keys())
    total_neurons_list.remove(target_neuron)

    """
    Example of how to find neurons within x column
    """

    # test 1
    y_width = y_height = 11000
    z_width = z_height = 11000
    x_width = x_height = 11000
    circular_radius = 13000
    
    # test 2 (more strict to try and keep around 10 neurons for at least 90 %)
    y_width = y_height = 6600
    z_width = z_height = 11900
    x_width = x_height = 2850
    circular_radius = 13700
    
    #test 3 (with equal volume) 
    y_new_width = 4385.621445508057
    z_new_width = 5469.83095905981
    x_new_width = 3281.8985754358864
    
    y_width = y_height = y_new_width
    z_width = z_height = z_new_width
    x_width = x_height = x_new_width
    
    
    width_axis = "x_distance"; height_axis = "z_distance"
    y_column_subgroup = find_column_subgroup(width_axis,height_axis,target_neuron,total_neurons_list,y_width,y_height)

    
    width_axis = "x_distance"; height_axis = "y_distance"
    z_column_subgroup = find_column_subgroup(width_axis,height_axis,target_neuron,total_neurons_list,z_width,z_height)

    
    width_axis = "y_distance"; height_axis = "z_distance"
    x_column_subgroup = find_column_subgroup(width_axis,height_axis,target_neuron,total_neurons_list,x_width,x_height)

    
    circular_subgroup = find_radius_subgroup(target_neuron,total_neurons_list,circular_radius)


    subgroups_list = dict(y_column_subgroup=y_column_subgroup,
                         z_column_subgroup=z_column_subgroup,
                         x_column_subgroup=x_column_subgroup,
                         circular_subgroup=circular_subgroup)

    n_random_shuffles = 1000


    
    for subgroup_name,restricted_neuron_group in subgroups_list.items():
        #get the mse of that subgroup
        restricted_neuron_group_mse = calc_mse(restricted_neuron_group,
                                              target_scaled)
        outside_neurons = [k for k in total_neurons_list if 
                       ((k not in restricted_neuron_group) and 
                        k != target_neuron)]
#         print("len(restricted_neuron_group) = " + str(len(restricted_neuron_group)))
#         print("len(outside_neurons) = " + str(len(outside_neurons)))
        
        random_shuffles_mse = []
        for i in range(0,n_random_shuffles):
            #get a random list from outside the subgroup
            if len(restricted_neuron_group) < len(outside_neurons):
                outside_neurons_rand = random.sample(outside_neurons,len(restricted_neuron_group))
            else:
                if i == 1:
                    print("not using sampling")
                outside_neurons_rand = outside_neurons
            
            if len(restricted_neuron_group) < len(outside_neurons_rand):
                print("RESTRICTED GROUP NUMBER IS LESS")
            random_shuffles_mse.append(calc_mse(outside_neurons_rand,
                                              target_scaled))

        #save the random shuffles
        mse_data[target_neuron][subgroup_name] = dict() 
        mse_data[target_neuron][subgroup_name]["shuffles"] = random_shuffles_mse
        mse_data[target_neuron][subgroup_name]["real_group"] = restricted_neuron_group_mse
        mse_data[target_neuron][subgroup_name]["real_group_len"] = len(restricted_neuron_group)
        
        probability_less_than = np.sum(random_shuffles_mse<restricted_neuron_group_mse)/len(random_shuffles_mse)
        mse_data[target_neuron][subgroup_name]["probability"] = probability_less_than
print(f"Total time for neurons = " + str(time.time() - start_time))


x_parameter_string = f"x_width = {x_width}, x_height = {x_height}"
y_parameter_string = f"y_width = {y_width}, y_height = {y_height}"
z_parameter_string = f"z_width = {z_width}, z_height = {z_height}"
sphere_parameter_string = f"circular_radius = {circular_radius}"


whole_parameters_string = "\n"+x_parameter_string+ \
                            "\n"+y_parameter_string+ \
                            "\n"+z_parameter_string+ \
                            "\n"+sphere_parameter_string

parameter_strings = [y_parameter_string,z_parameter_string,x_parameter_string,
                    sphere_parameter_string]

whole_parameters_string

#get all of the probabilities for each group

column_sphere_names = subgroups_list.keys()
total_probabilities = []
total_group_number = []
column_sphere_subgroup_number = dict()
column_sphere_probabilities = dict()
for volume_name,param in zip(column_sphere_names,parameter_strings):
    column_sphere_probabilities[volume_name] = []
    column_sphere_subgroup_number[volume_name] = []
    for a in mse_data.keys():
            column_sphere_probabilities[volume_name].append(mse_data[a][volume_name]["probability"])
            column_sphere_subgroup_number[volume_name].append(mse_data[a][volume_name]["real_group_len"])
    plt.figure()
    plt.hist(column_sphere_probabilities[volume_name],bins=100)
    mean_group_number = np.mean(column_sphere_subgroup_number[volume_name])
    std_dev_group_number = np.std(column_sphere_subgroup_number[volume_name])
    plt.title(volume_name  + " probability distributions\n" + param 
                           + "\nmean subgroup number = " + str(np.round(mean_group_number,3))
                           + "\nstd dev subgroup number = " + str(np.round(std_dev_group_number,3))
                            + "\n Mean probability = " + str(np.round(np.mean(column_sphere_probabilities[volume_name]),3)))
    total_group_number += column_sphere_subgroup_number[volume_name]
    total_probabilities += column_sphere_probabilities[volume_name]
    
    
mean_group_number = np.mean(total_group_number)
std_dev_group_number = np.std(total_group_number)

plt.figure()
plt.hist(total_probabilities,bins=100)
plt.title("Total probability distributions" + whole_parameters_string
         + "\nmean subgroup number = " + str(np.round(mean_group_number,3))
         + "\nstd dev subgroup number = " + str(np.round(std_dev_group_number,3))
         + "\n Mean probability = " + str(np.round(np.mean(total_probabilities),3)))

# ----- 2nd Part: Start graphing the heat maps according to the locations in the volume --

# Radius Clustering

In [ ]:
#Shows clustering of the circular subgroups

y_col_values = [mse_data[a]["circular_subgroup"]["probability"] for a in soma_cell_location_dict.keys()]
soma_centers_list =  np.array([soma_cell_location_dict[a]["location"] for a in soma_cell_location_dict.keys()])

fig = plt.figure(figsize=(16,4))
fig.tight_layout()
ax = plt.subplot(1,3,1)
sc = ax.scatter(soma_centers_list[:,0],
           soma_centers_list[:,1], 
           c=y_col_values, 
           marker='o')

ax.set_xlabel('x axis')
ax.set_ylabel('Y axis')
plt.colorbar(sc)

ax = plt.subplot(1,3,2)

sc = ax.scatter(soma_centers_list[:,0],
           soma_centers_list[:,2], 
           c=y_col_values, 
           marker='o')

ax.set_xlabel('x axis')
ax.set_ylabel('z axis')
plt.colorbar(sc)

ax = plt.subplot(1,3,3)
sc = ax.scatter(soma_centers_list[:,1],
           soma_centers_list[:,2], 
           c=y_col_values, 
           marker='o')

ax.set_xlabel('y axis')
ax.set_ylabel('z axis')
plt.colorbar(sc)


#soma_cell_location_dict

from matplotlib import pyplot
from mpl_toolkits.mplot3d import Axes3D
import random


fig = pyplot.figure()
ax = Axes3D(fig)

sc = ax.scatter(soma_centers_list[:,0], soma_centers_list[:,1], soma_centers_list[:,2],c=y_col_values)
ax.set_xlabel("X axis")
ax.set_xlabel("Y axis")
ax.set_xlabel("Z axis")
plt.colorbar(sc)
pyplot.show()

# Y Column Clustering

In [ ]:
#Total groups = ['y_column_subgroup', 'z_column_subgroup', 'x_column_subgroup', 'circular_subgroup']
y_col_values = [mse_data[a]["y_column_subgroup"]["probability"] for a in soma_cell_location_dict.keys()]
soma_centers_list =  np.array([soma_cell_location_dict[a]["location"] for a in soma_cell_location_dict.keys()])

fig = plt.figure(figsize=(16,4))
fig.tight_layout()
ax = plt.subplot(1,3,1)
sc = ax.scatter(soma_centers_list[:,0],
           soma_centers_list[:,1], 
           c=y_col_values, 
           marker='o')

ax.set_xlabel('x axis')
ax.set_ylabel('Y axis')


ax = plt.subplot(1,3,2)

sc = ax.scatter(soma_centers_list[:,0],
           soma_centers_list[:,2], 
           c=y_col_values, 
           marker='o')

ax.set_xlabel('x axis')
ax.set_ylabel('z axis')


ax = plt.subplot(1,3,3)
sc = ax.scatter(soma_centers_list[:,1],
           soma_centers_list[:,2], 
           c=y_col_values, 
           marker='o')

ax.set_xlabel('y axis')
ax.set_ylabel('z axis')
plt.colorbar(sc)


#soma_cell_location_dict
from matplotlib import pyplot
from mpl_toolkits.mplot3d import Axes3D
import random


fig = pyplot.figure()
ax = Axes3D(fig)

sc = ax.scatter(soma_centers_list[:,0], soma_centers_list[:,1], soma_centers_list[:,2],c=y_col_values)
ax.set_xlabel("X axis")
ax.set_xlabel("Y axis")
ax.set_xlabel("Z axis")
plt.colorbar(sc)
pyplot.show()

# Z Column Clustering

In [ ]:
#Total groups = ['y_column_subgroup', 'z_column_subgroup', 'x_column_subgroup', 'circular_subgroup']
y_col_values = [mse_data[a]["z_column_subgroup"]["probability"] for a in soma_cell_location_dict.keys()]
soma_centers_list =  np.array([soma_cell_location_dict[a]["location"] for a in soma_cell_location_dict.keys()])

fig = plt.figure(figsize=(16,4))
fig.tight_layout()
ax = plt.subplot(1,3,1)
sc = ax.scatter(soma_centers_list[:,0],
           soma_centers_list[:,1], 
           c=y_col_values, 
           marker='o')

ax.set_xlabel('x axis')
ax.set_ylabel('Y axis')


ax = plt.subplot(1,3,2)

sc = ax.scatter(soma_centers_list[:,0],
           soma_centers_list[:,2], 
           c=y_col_values, 
           marker='o')

ax.set_xlabel('x axis')
ax.set_ylabel('z axis')


ax = plt.subplot(1,3,3)
sc = ax.scatter(soma_centers_list[:,1],
           soma_centers_list[:,2], 
           c=y_col_values, 
           marker='o')

ax.set_xlabel('y axis')
ax.set_ylabel('z axis')
plt.colorbar(sc)


from matplotlib import pyplot
from mpl_toolkits.mplot3d import Axes3D
import random


fig = pyplot.figure()
ax = Axes3D(fig)

sc = ax.scatter(soma_centers_list[:,0], soma_centers_list[:,1], soma_centers_list[:,2],c=y_col_values)
ax.set_xlabel("X axis")
ax.set_xlabel("Y axis")
ax.set_xlabel("Z axis")
plt.colorbar(sc)
pyplot.show()

#soma_cell_location_dict

# X column clustering

In [ ]:
#Total groups = ['y_column_subgroup', 'z_column_subgroup', 'x_column_subgroup', 'circular_subgroup']
y_col_values = [mse_data[a]["x_column_subgroup"]["probability"] for a in soma_cell_location_dict.keys()]
soma_centers_list =  np.array([soma_cell_location_dict[a]["location"] for a in soma_cell_location_dict.keys()])

fig = plt.figure(figsize=(16,4))
fig.tight_layout()
ax = plt.subplot(1,3,1)
sc = ax.scatter(soma_centers_list[:,0],
           soma_centers_list[:,1], 
           c=y_col_values, 
           marker='o')

ax.set_xlabel('x axis')
ax.set_ylabel('Y axis')


ax = plt.subplot(1,3,2)

sc = ax.scatter(soma_centers_list[:,0],
           soma_centers_list[:,2], 
           c=y_col_values, 
           marker='o')

ax.set_xlabel('x axis')
ax.set_ylabel('z axis')


ax = plt.subplot(1,3,3)
sc = ax.scatter(soma_centers_list[:,1],
           soma_centers_list[:,2], 
           c=y_col_values, 
           marker='o')

ax.set_xlabel('y axis')
ax.set_ylabel('z axis')
plt.colorbar(sc)

from matplotlib import pyplot
from mpl_toolkits.mplot3d import Axes3D
import random


fig = pyplot.figure()
ax = Axes3D(fig)

sc = ax.scatter(soma_centers_list[:,0], soma_centers_list[:,1], soma_centers_list[:,2],c=y_col_values)
ax.set_xlabel("X axis")
ax.set_xlabel("Y axis")
ax.set_xlabel("Z axis")
plt.colorbar(sc)
pyplot.show()

#soma_cell_location_dict

In [ ]:
circular_subgroup

mse_data[a][volume_name]["probability"]
volume_name

plt.scatter(x=x, y=y, c=value)

# Find what a distribution looks like where subgroup has probability of 1 (put on hold for right now):

In [ ]:
# how to get the distributions from one of the shuffles
target_neuron = 648518346341371119
print(target_neuron)
currnet_subgroup = "circular_subgroup"
mse_current_target_shuffles = mse_data[target_neuron][currnet_subgroup]["shuffles"]

mse_current_target_real  = mse_data[target_neuron][currnet_subgroup]["real_group"]
probability_less_than = np.sum(mse_current_target_shuffles<mse_current_target_real)/len(mse_current_target_shuffles)
plt.figure()
plt.hist(mse_current_target_shuffles,bins= 40)
print("mse_current_target_real = " + str(mse_current_target_real))
print("probability_less_than = " + str(probability_less_than))

In [ ]:
# finding how all of the probabilities change as you scale the volumes

# Shuffling for validation: 

In [ ]:
print("hi")

In [ ]:
import random

outside_neurons = [k for k in total_neurons_list if 
                   ((k not in restricted_neuron_group) and 
                    k != target_neuron)]
#checking that correct
#print(np.sum([len(outside_neurons),len(restricted_neuron_group),1]))

# get a random shuffling of the outside neurons
outside_neurons_rand = random.sample(outside_neurons,len(restricted_neuron_group))
print(len(outside_neurons_rand))
outside_neurons_rand

In [ ]:
"""
Need to shift the neurons so we can get the 
get the list of global neurons that are not the specific neuron or subset
"""

target_neuron = 648518346341371119
target_neuron_orientation_preferene = seg_id_to_orientation[target_neuron]

"""
Example of how to find neurons within a certain radius
"""
total_neurons_list = list(soma_cell_location_dict.keys())
total_neurons_list.remove(target_neuron)

#get total list of orientations without the target neuron
total_neurons_list_orientation_preference = np.array([seg_id_to_orientation[k] for k in total_neurons_list])
len(total_neurons_list_orientation_preference)

#now to get the total list shifted by subtracting off the target neuron
target_scaled = target_neuron_orientation_preferene*2
total_neurons_scaled = total_neurons_list_orientation_preference*2

differences = pycs.pairwise_cdiff(total_neurons_scaled,target_scaled)/2
squared_difference = np.sum(differences**2)

#store this as an outcome in the overall probability distribution

    

In [ ]:
differences[1]**2

In [ ]:
"""    just making sure computation was equivalent   """
# differences_check = np.array([cdiff(k,target_scaled) for k in total_neurons_scaled])/2
# print((len(differences_check),np.min(differences_check),np.max(differences_check)))

# for d,cd in zip(differences,differences_check):
#     if abs(d-cd) > 0.0001:
#         print("not equal")